## Easy

Построить прогноз по одной марке машин при помощи решающих деревьев, взять только числовые признаки. В качестве метрики взять `r2_score`. Оценить важность признаков, проинтерпретировать модель.

## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

## Hard

Собрать датасет по одной модели на любом агрегаторе объявлений (модель должна быть из датасета). Сравнить какие признаки наиболее важны на нашем рынке, по сравнению с рынком в UK.

Сервисы не любят парсеры, поэтому скорее всего в автоматическом режиме не получится распарсить много данных. По-крайней мере, у меня не получилось как-то быстро обойти капчу. Поэтому:

- Можно вручную вбить в эксельку данные по нескольким десяткам машин
- Можно скачать несколько html и написать скрипт для их парсинга (можно использовать beautiful soup) - в таком подходе ожидаю больше сотни примеров
- Можно найти агрегатор без капчи или апишку (примеры - телеграм каналы, VK API) и собрать данные оттуда

## Easy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pandas as pd


In [2]:
data=pd.read_csv("audi.csv",sep=',')


In [3]:
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

X = data.drop(columns=["price", "model", "transmission", "fuelType"])
y = data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model=DecisionTreeRegressor(random_state=42)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
r2

0.9134150265264636

In [4]:
pd.DataFrame(zip(X_train.columns, model.feature_importances_))

,0,1
0,year,0.239486
1,mileage,0.066148
2,tax,0.039614
3,mpg,0.462504
4,engineSize,0.192247


## Выводы
Можно заметить,что при выборе автомобиля важным признаком является расход топлива(mpg). Также важными факторами является год производства(year) и объем двигателя(engineSize).
Однако налоги и пробег не являются важными факторами.
Получена метрика 0.91

## Normal

In [5]:
data_audi=pd.read_csv("audi.csv")
data_bmw=pd.read_csv("bmw.csv")
data_cclass=pd.read_csv("cclass.csv")
data_focus=pd.read_csv("focus.csv")
data_hyundi = pd.read_csv("hyundi.csv")
data_hyundi["tax"]=data_hyundi["tax(£)"]
data_hyundi.drop(columns=["tax(£)"], inplace=True)
data_merc = pd.read_csv("merc.csv")
data_skoda = pd.read_csv("skoda.csv")
data_toyota = pd.read_csv("toyota.csv")
data_vauxhall = pd.read_csv("vauxhall.csv")
data_vw = pd.read_csv("vw.csv")

In [6]:
data_hyundi

,model,year,price,transmission,mileage,fuelType,mpg,engineSize,tax
0,I20,2017,7999,Manual,17307,Petrol,58.9,1.2,145
1,Tucson,2016,14499,Automatic,25233,Diesel,43.5,2.0,235
2,Tucson,2016,11399,Manual,37877,Diesel,61.7,1.7,30
3,I10,2016,6499,Manual,23789,Petrol,60.1,1.0,20
4,IX35,2015,10199,Manual,33177,Diesel,51.4,2.0,160
...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,78.4,1.6,0
4856,I40,2015,7830,Manual,59508,Diesel,65.7,1.7,30
4857,I10,2017,6830,Manual,13810,Petrol,60.1,1.0,20
4858,Tucson,2018,13994,Manual,23313,Petrol,44.8,1.6,145


In [7]:
dfs = [data_audi, data_bmw, data_cclass, data_focus, data_hyundi, data_merc, data_skoda, data_toyota, data_vauxhall, data_vw]
merged_data = pd.concat(dfs, ignore_index=True)
print(merged_data)

      model  year  price transmission  mileage fuelType    tax   mpg  \
0        A1  2017  12500       Manual    15735   Petrol  150.0  55.4   
1        A6  2016  16500    Automatic    36203   Diesel   20.0  64.2   
2        A1  2016  11000       Manual    29946   Petrol   30.0  55.4   
3        A4  2017  16800    Automatic    25952   Diesel  145.0  67.3   
4        A3  2019  17300       Manual     1998   Petrol  145.0  49.6   
...     ...   ...    ...          ...      ...      ...    ...   ...   
90570   Eos  2012   5990       Manual    74000   Diesel  125.0  58.9   
90571   Fox  2008   1799       Manual    88102   Petrol  145.0  46.3   
90572   Fox  2009   1590       Manual    70000   Petrol  200.0  42.0   
90573   Fox  2006   1250       Manual    82704   Petrol  150.0  46.3   
90574   Fox  2007   2295       Manual    74000   Petrol  145.0  46.3   

       engineSize  
0             1.4  
1             2.0  
2             1.4  
3             2.0  
4             1.0  
...           .

In [8]:
dummy_df = pd.get_dummies(data=merged_data, columns=["transmission", "fuelType", "model"], drop_first=True)
dummy_df = dummy_df.dropna()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dummy_df.drop(columns=["price"]), dummy_df["price"],test_size=0.2)
X_train.head()

,year,mileage,tax,mpg,engineSize,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Electric,fuelType_Hybrid,...,model_ Z3,model_ Z4,model_ Zafira,model_ Zafira Tourer,model_ i3,model_ i8,model_180,model_200,model_220,model_230
88539,2015,63004,20.0,55.4,2.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14405,2019,7729,150.0,40.4,1.5,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5914,2019,8499,145.0,48.7,2.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
55794,2017,28952,20.0,55.4,2.5,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
19144,2018,14449,145.0,48.7,2.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Поскольку 2^17 больше размера данных,то хватит глубины 17.

In [10]:
dt_params = {
    'max_depth': [5, 7, 10, 12, 15, 16, 17],
    'max_leaf_nodes': [None, 10, 25, 50, 100,500]
}

In [11]:
model=DecisionTreeRegressor()

In [12]:
from sklearn.model_selection import RandomizedSearchCV
search = RandomizedSearchCV(model, dt_params, scoring='r2')
search.fit(X_train, y_train)
search.best_estimator_

DecisionTreeRegressor(max_depth=15)

In [13]:
model=DecisionTreeRegressor(max_depth=15)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
r2 = r2_score(y_test, y_pred)
r2

0.9299646931382162

In [14]:
from scipy.stats import ttest_ind

splits_test_scores = [search.cv_results_[f"split{i}_test_score"] for i in range(5)]
cvs = tuple(zip(*splits_test_scores))

for n in range(10):
    models_beated = len([ttest_ind(cvs[n], cvs[i], alternative='greater')[1] for i in range(10) if ttest_ind(cvs[n], cvs[i], alternative='greater')[1] <= 0.05])
    print(f"Model {n} better than {models_beated} models.")

Model 0 better than 8 models.
Model 1 better than 6 models.
Model 2 better than 1 models.
Model 3 better than 1 models.
Model 4 better than 0 models.
Model 5 better than 7 models.
Model 6 better than 1 models.
Model 7 better than 1 models.
Model 8 better than 1 models.
Model 9 better than 9 models.


Тест Стьюдента выдал,что модели 9 лучше других